### Preparing the environment / Preparando o ambiente

In [65]:
# Importing Libraries / Importanto bibliotecas
import pandas as pd
from google.cloud import bigquery
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import geopandas as gp
import geojson


In [66]:
# Declaring the query / Declarando a consulta
query = """
    SELECT * FROM `bigquery-public-data.chicago_crime.crime` LIMIT 100000
"""

In [67]:
# Stating the GCP project / Declarando o projeto do GCP
bqclient = bigquery.Client(project="sz-lab-bq-2023-sandbox")

c:\Users\JPCONCEICAO\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\auth\_default.py:78: UserWarning:

Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 



In [68]:
query_job = bqclient.query(query) # Running the query we declared before / Executando a consulta que declaramos antes

In [69]:
# Then, we request the 'result' attribute, as the query response will be placed there
# A seguir solicitamos o atributo 'resultado', pois a resposta da consulta será colocada lá
resultado = query_job.result() 

In [70]:
resultado

In [71]:
df = resultado.to_dataframe()

In [72]:
df.head(2)

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,24030,JB325879,2018-06-28 07:02:00+00:00,106XX S LAFAYETTE AVE,0110,HOMICIDE,FIRST DEGREE MURDER,HOUSE,True,False,...,34,49,01A,1177815.0,1834251.0,2018,2022-09-18 04:45:51+00:00,41.700502,-87.624523,"(41.700501645, -87.624523155)"
1,19706,HS649337,2010-12-07 07:35:00+00:00,104XX S MICHIGAN AVE,0110,HOMICIDE,FIRST DEGREE MURDER,ABANDONED BUILDING,False,False,...,9,49,01A,1178889.0,1835859.0,2010,2022-09-18 04:45:51+00:00,41.704890,-87.620542,"(41.704889896, -87.620541865)"


### Data Cleaning

In [73]:
#Verifying if there is null values / Verificando se há valores nulos
df.isna().sum()

unique_key                 0
case_number                0
date                       0
block                      0
iucr                       0
primary_type               0
description                0
location_description     101
arrest                     0
domestic                   0
beat                       0
district                   0
ward                    8730
community_area          8735
fbi_code                   0
x_coordinate            1105
y_coordinate            1105
year                       0
updated_on                 0
latitude                1102
longitude               1102
location                1102
dtype: int64

In [74]:
# Droping null values/ Excluindo valores nulos
df = df.dropna()

In [75]:
#Checking again / Checando novamente
df.isna().sum()

unique_key              0
case_number             0
date                    0
block                   0
iucr                    0
primary_type            0
description             0
location_description    0
arrest                  0
domestic                0
beat                    0
district                0
ward                    0
community_area          0
fbi_code                0
x_coordinate            0
y_coordinate            0
year                    0
updated_on              0
latitude                0
longitude               0
location                0
dtype: int64

### Data Mining / Mineração de Dados

### 1. What are the 5 most recurring crimes? / Quais são os 5 crimes mais recorrentes?

In [76]:
df.head(2)

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,24030,JB325879,2018-06-28 07:02:00+00:00,106XX S LAFAYETTE AVE,0110,HOMICIDE,FIRST DEGREE MURDER,HOUSE,True,False,...,34,49,01A,1177815.0,1834251.0,2018,2022-09-18 04:45:51+00:00,41.700502,-87.624523,"(41.700501645, -87.624523155)"
1,19706,HS649337,2010-12-07 07:35:00+00:00,104XX S MICHIGAN AVE,0110,HOMICIDE,FIRST DEGREE MURDER,ABANDONED BUILDING,False,False,...,9,49,01A,1178889.0,1835859.0,2010,2022-09-18 04:45:51+00:00,41.704890,-87.620542,"(41.704889896, -87.620541865)"


In [77]:
#Creating the clustered database / Criando a base de dados agrupados
grouped = df.groupby(['primary_type'])
counts = grouped.size()
counts = counts.sort_values(ascending=False)
counts = counts[0:5].to_frame(name='Count').reset_index()
print(counts)

      primary_type  Count
0            THEFT  21216
1          BATTERY  15754
2  CRIMINAL DAMAGE  10582
3        NARCOTICS   7007
4          ASSAULT   5653


In [78]:
# Sorting by score / Ordenando pela contagem
counts = counts.sort_values(by='Count', ascending=True)

In [79]:
# Plotting the chart / Plotando o gráfico
fig = px.bar(counts, x="Count", y="primary_type", orientation='h',  width=650, height=400)
fig.show()

### 2. What time do these crimes happen more?
#### 2. Em que horários os crimes ocorrem mais?

In [80]:
# Plotting the chart / Plotando o gráfico
fig = px.line(df.groupby(df['date'].dt.hour)['unique_key'].count().reset_index(), x="date", y="unique_key", 
              title='Crime occurrence times',  width=800, height=400)
fig.show()

### 3. Is the incidence of crimes growing up or decreasing over the years?
#### 3. A incidência de crimes está crescendo ou diminuindo ao longo dos anos?

In [127]:
# Plotting the chart / Plotando o gráfico
fig = px.line(df.groupby(df['date'].dt.year)['unique_key'].count().reset_index(), x="date", y="unique_key",
               title='Crime occurrence per year',  width=1000, height=500)
fig.show()

### 4. Proportionately to their occurence, which crimes had most effectiveness in arrest?
#### 4. Porporcionalmente a suas ocorrencias, quais crimes tiveram mais efetividade de prisão?

In [82]:
# Grouping by count per class / Agrupando por contagem por classe
df_arrest = df.groupby(['primary_type','arrest'])['unique_key'].count().reset_index()

In [83]:
df_arrest.head()

,primary_type,arrest,unique_key
0,ARSON,False,136
1,ARSON,True,24
2,ASSAULT,False,4402
3,ASSAULT,True,1251
4,BATTERY,False,11978


In [84]:
# Calculating the relative percentage of each class in each categories
# Calcular a porcentagem relativa de cada classe em cada categoria
relative_perc = ((df.groupby('primary_type')['arrest'].value_counts() / df.groupby('primary_type')['arrest'].value_counts().groupby('primary_type').sum()) * 100).rename('%_rel').reset_index()


# Printing the result/ Imprimir o resultado
relative_perc.head()

,primary_type,arrest,%_rel
0,ARSON,False,85.000000
1,ARSON,True,15.000000
2,ASSAULT,False,77.870157
3,ASSAULT,True,22.129843
4,BATTERY,False,76.031484


In [85]:
# Filtering just the rows where the arrest ocurred / Criando um DF apenas com as linhas em que houve a prisão
relative_perc_true = relative_perc[relative_perc['arrest']==True].sort_values(by='%_rel', ascending= False)

In [86]:
# Plotting the chart / Plotando o gráfico
fig = px.bar(relative_perc_true, x="primary_type", y= '%_rel',title='Arrest relative %')
fig.update_layout(
    title="Arrest relative %",
    xaxis_title="Crime",
    yaxis_title=r"arrest %"
)
fig.show()

##### 5. Which are the 5 local where crimes occur the most and what are their relative percentage comparing to the total of occurences?
###### 5. Quais os 5 tipos de locais onde há maior ocorrencia de crimes e qual a porcentagem relativa comparando com o total de ocorrências?

In [87]:
#Verifying the classes / Verificando as classes
df_tl = df['location_description'].value_counts(); df_tl

STREET               22519
RESIDENCE            15329
APARTMENT             8439
SIDEWALK              8132
OTHER                 3323
                     ...  
BOAT / WATERCRAFT        1
HOTEL                    1
OFFICE                   1
DRIVEWAY                 1
TAVERN                   1
Name: location_description, Length: 152, dtype: int64

In [117]:
# Creating the data frame which will show us the representativeness of each type of crime
# Criando o DF que irá nos mostrar a representatividade de cada tipo de crime
df_top_locals = ((df.groupby('location_description').value_counts()/(df.groupby('location_description').value_counts().sum()))*100).rename('%_rel').reset_index()
df_top_locals = df_top_locals.groupby('location_description')['%_rel'].sum().reset_index().sort_values('%_rel', ascending=False).reset_index().drop(['index'],axis=1)

In [121]:
df_top_locals.head(10)

,location_description,%_rel
0,STREET,24.936051
1,RESIDENCE,16.974321
2,APARTMENT,9.344791
3,SIDEWALK,9.004839
4,OTHER,3.679670
5,PARKING LOT/GARAGE(NON.RESID.),2.924469
6,SMALL RETAIL STORE,2.298825
7,RESTAURANT,2.086217
8,RESIDENCE-GARAGE,2.055212
9,"SCHOOL, PUBLIC, BUILDING",1.940049


In [123]:
# Lets separate the top five of the other classes/ Vamos separar o top 5 das outras classes
df_top_locals.loc[4:len(df_top_locals), 'location_description'] = 'OTHERS'

In [124]:
df_top_locals

,location_description,%_rel
0,STREET,24.936051
1,RESIDENCE,16.974321
2,APARTMENT,9.344791
3,SIDEWALK,9.004839
4,OTHERS,3.679670
...,...,...
147,OTHERS,0.001107
148,OTHERS,0.001107
149,OTHERS,0.001107
150,OTHERS,0.001107


In [125]:
# Plotting the chart / Plotando o gráfico
fig = px.pie(df_top_locals, values='%_rel', names='location_description', color = 'location_description',
              color_discrete_map={'OTHERS':'RGB(115,111,76)',
                                 'SIDEWALK':'#511CFB',
                                 'APARTMENT':'royalblue',
                                 'RESIDENCE': '#1616A7' ,
                                 'STREET': '#0D2A63'},   width=650, height=400)
fig.update_layout(title_text='Places with highest incidence of crime', title_x=0.4)
fig.show()

#### 6. Draw a heatmap according to the incidence of crimes / Monte um mapa de calor de acordo com a incidência de crimes;

In [92]:
# importing the library / Importando a biblioteca
import folium
from folium.plugins import HeatMap

In [126]:
# Group the coordinates by frequency / Agrupando as coordenadas pela frequencia
freq = df.groupby(['latitude', 'longitude']).size().reset_index(name='count')

# Create a map centered on the mean of the coordinates / Criando um mapa centralizado na média das coordendas
center = [df['latitude'].mean(), df['longitude'].mean()]
m = folium.Map(location=center, zoom_start=5)

# Create a heat map layer with the frequency data / Criando o mapa de calor com os dados de frequencia
heat_data = [[row['latitude'], row['longitude'], row['count']] for index, row in freq.iterrows()]
HeatMap(heat_data).add_to(m)

# Display the map / Mostrar o mapa
m


### Insights

- As we can see, the criminal rate grows rapidly after 7:00;
  Unfortunately, the database doesn't provide us the information if this time is A.M. or P.M;
- The more recurrent class of crime was THEFT, and its arrest rate is just 11,34 %;
- The main places where crimes happened was in the streets;
- After 2003, the crime rate decreased quickly.

- Notamos que a taxa de criminalidade cresce rapidamente após as 7:00;
  Infelizmente, o banco de dados não nos fornece a informação se esta hora é A.M. ou PM;
- A classe de crime mais recorrente foi o ROUBO, e sua taxa de prisão é de apenas 11,34%;
- Os principais locais onde ocorreram os crimes foram as ruas;
- Após 2003, a taxa de criminalidade diminuiu rapidamente.